<img src='./images/Rectangle.svg'/>


- **<u>Mo</u>**lecular **<u>S</u>**imulation **<u>De</u>**sign **<u>F</u>**ramework, or MoSDeF, is a robust Python-based, open-source software framework designed to faciliate the initialization, atom-typing, and screeening of soft matter system using molecular dynamics simulation. The project was developed initially at Vanderbilt University, in collaboration with software engineer from the Institute for Software Integrated System (ISIS). The project later expanded into a multi-university collaboration with Vanderbilt University being the lead institute. 

- The MoSDeF software suite comprises several libraries, namely `mbuild`, `foyer`, and `gmso`, each targets a different component of system initialization. Specifically, `mbuild` library can be used to systematically construct any molecular systems, either atomistic or coarse-grained, while `foyer` can be used to atom-type (assign type and parameters) to all particles, bonds, angles, and dihedrals in the created system. The `gmso` library, which is still under development, will be the main data structure that can be used to store all the information of the system, including the details of the system (particles and their positions) and parameters of all the atom types, bond types, angle types, and dihedral types. 

- By creating tools that allow user to easily put together complicated system and automating "trivial" but rather tedius steps (such as writing out to simulation engine specific molecular files), MoSDeF allows user to focus on system design, be able to build more (quantity-wise) and complicated systems. This does not increase the capability of molecular research, especially screening studies, where research iterate over a wide variable space (so many of unqiue systems), but also improve the overall reprodicibility by minimizing human errors in most error proning steps.

## This Poster

- In this poster, we will try to demonstrate how MoSDeF can be used in a molecular simulation workflow. Along the way we will highlight important features of each package. However, in context of a short demonstration, we will only do a rather simple simulation (equilibrium simulation of a box of small molecules). We will include examples of more complicated example/tutorials, which were created based on published work but has been simplified for pegodagy/demonstration purpose.

### mBuild

- `mBuild` is MoSDeF's molecular building library within MoSDeF. The library is hosted on github (https://github.com/mosdef-hub/mbuild), including a series of tutorials showcasing its importance feature and use-cases. 
- In simple term, this library included features that allow user to create a molecular system in various way: loading from different files format, from SMILES string (a 2D representation of molecule), or building molecular system starting from a particle to a molcules, and stacking those in a hierarchy manner. Users can also create their own class of molecular system, and added back as an extension to the core `mbuild` library. 

- Create a molecule from a SMILES string is very handy tool go get create most small molecules. A SMILES string is a 2D representation of molecule, the SMILES string of most small molecules can be found in public database, such as https://pubchem.ncbi.nlm.nih.gov/ 
- We have a small demonstration below, you can try enter a SMILES string and our `mbuild` library will read it in, turn it into a `mbuild.Compound` and visualize it! (If you don't have one in mind, try "C1CCCCC1")

In [ ]:
import src.widget_events as widgets
import ipywidgets as iwidgets

display(widgets.smiles_box)
display(widgets.out_smiles)

- In this example, particularly, we will demonstrate a simple process of loading a small molecules from file, and used that to build a box of molecules, where user can specify the size of the box and the number of particles in the box. 

In [ ]:
display(widgets.compound_dropdown)
display(widgets.out_mol)

In [ ]:
display(iwidgets.VBox([widgets.box_slider, widgets.n_slider]))
display(widgets.out_box)

- Due to restriction of forcefield parameters availability, we will continue our example workflow with the compound selected from the drop down list.

## Foyer

`Foyer` is the atom-typing and parameterization package within the MoSDeF suite of tools.  In molecular simulation, the bonded and non-bonded parameters that define the interactions between particles is called a Force Field.  These interaction parameters are derived by fitting to a specific physical property, such as density.

In `Foyer`, force field information is contained within an XML file, that can be loaded in as a `Forcefield` object.  By calling `Forcefield.apply()` on an `mbuild.Compound`, the force field parameters are automatically applied to the compound, returning a fully parametrized system.

Below we will demonstrate the process of initializing a `Forcefield` object with foyer, and applying the force field parameters to the `mBuild.Compound` initialized above.  We will use the General Amber Force Field (GAFF) using the `GAFF-foyer` plugin, and we will assign partial charges to each atom by using the `antefoyer` plugin.

In [ ]:
import foyer
import antefoyer

gaff_ff = foyer.forcefields.load_GAFF()

typed_compound = gaff_ff.apply(widgets.BOX_OF_COMPOUNDS,
                              assert_dihedral_params=False)

We have written a small function to help apply the partial charges with `antefoyer` and `antechamber`.  The charge assignment method we are using is `AM1BCC`, and we are setting the net charge to `0.0`.

In [ ]:
def apply_charges(box_structure, single_compound, n_atoms, ff):
    single_mol_struct = ff.apply(single_compound)
    single_mol_struct_charge = antefoyer.ante_charges(
            single_mol_struct, 'bcc', net_charge=0.00, multiplicity=1)
    
    for index, atom in enumerate(box_structure.atoms):
        atom.charge = single_mol_struct_charge.atoms[index%n_atoms].charge
    return box_structure

In [ ]:
charge_structure = apply_charges(box_structure=typed_compound,
                                 single_compound=widgets.COMPOUND,
                                 n_atoms=widgets.COMPOUND.n_particles,
                                 ff=gaff_ff)

## GMSO